## Set catalog and schema

We set the catalog and schema to organise our data and ensure it is stored in the correct location. Change these to suit your workspace.

In [0]:
CATALOG = "marcell"
SCHEMA = "ray_multimodel_inference"

Create catalog, schema and volume if they don't exist, and create directories for compressed, raw audio files and models.

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.data")
dbutils.fs.mkdirs(f"/Volumes/{CATALOG}/{SCHEMA}/data/compressed/LJSpeech")
dbutils.fs.mkdirs(f"/Volumes/{CATALOG}/{SCHEMA}/data/raw_audio/LJSpeech")
dbutils.fs.mkdirs(f"/Volumes/{CATALOG}/{SCHEMA}/data/models")

## Download models from Hugging Face

We download two models from Hugging Face. We do this because it's more efficient to download these larger models once and retrieve them from storage for every batch of inference:
- [Whisper-medium](https://huggingface.co/openai/whisper-medium)
- [Phi-4](https://huggingface.co/microsoft/phi-4)

In [0]:
from transformers import pipeline
import torch

Whisper-medium is a state-of-the-art automatic speech recognition (ASR) model developed by OpenAI. It is designed to transcribe spoken language into written text with high accuracy.

In [0]:
WHISPER_MODEL_SAVE_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/data/models/whisper-medium/"

dbutils.fs.mkdirs(WHISPER_MODEL_SAVE_PATH)

whisper_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium",
    torch_dtype=torch.float16,
    device="cuda:0"
)

whisper_pipeline.save_pretrained(WHISPER_MODEL_SAVE_PATH)

Phi-4 is a state-of-the-art language model developed by Microsoft. It is designed for text generation and can be used for various natural language processing tasks. We will use it for simple classification.

In [0]:
PHI_MODEL_SAVE_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/data/models/phi-4/"

dbutils.fs.mkdirs(PHI_MODEL_SAVE_PATH)

phi_pipeline = pipeline(
    "text-generation",
    model="microsoft/phi-4",
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto",
)

phi_pipeline.save_pretrained(PHI_MODEL_SAVE_PATH)